In [6]:
import gymnasium as gym
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from stable_baselines3 import DQN, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common.callbacks import (
    EvalCallback,
    StopTrainingOnRewardThreshold,
    BaseCallback,
)
from stable_baselines3.common.evaluation import evaluate_policy

In [7]:
# Create the LunarLander-v3 environment
env = gym.make(
    "LunarLander-v3",
    # continuous=False,
    # gravity=-10.0,
    # enable_wind=False,
    # wind_power=15.0,
    # turbulence_power=1.5,
    render_mode="rgb_array"
)

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)

eval_callback = EvalCallback(
    env,
    best_model_save_path="../logs/exercise_2/ppo/",
    log_path="../logs/exercise_2/ppo/",
    eval_freq=5_000,
    deterministic=True,
    render=False,
    n_eval_episodes=50,
    callback_on_new_best=stop_callback,
)

In [ ]:
# Create PPO model
model = PPO(
    "MlpPolicy",
    env,
    # learning_rate=1e-3,
    seed=42,
    verbose=1,
    tensorboard_log="../logs/exercise_2/ppo/ppo_tensorboard/",
    device="cpu",
)
# model = PPO( # TODO fix
#     "CnnPolicy",
#     env,
#     seed=42,
#     verbose=1,
#     tensorboard_log="../logs/exercise_2/ppo_tensorboard/",
#     device="cuda",
# )

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [10]:
# Train model on the environment
TRAINING_TIMESTEPS = 1_000_000
model.learn(total_timesteps=TRAINING_TIMESTEPS, callback=[eval_callback])

Logging to ../logs/exercise_2/ppo_tensorboard/PPO_11
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 86.7     |
|    ep_rew_mean     | -165     |
| time/              |          |
|    fps             | 3746     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 91.3        |
|    ep_rew_mean          | -155        |
| time/                   |             |
|    fps                  | 2179        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007234047 |
|    clip_fraction        | 0.00601     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_vari

/home/javier/.cache/pypoetry/virtualenvs/deep-reinforcement-learning-gymnasium-u3px5S1O-py3.13/lib/python3.13/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=5000, episode_reward=-874.88 +/- 518.98
Episode length: 176.80 +/- 56.59
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 177         |
|    mean_reward          | -875        |
| time/                   |             |
|    total_timesteps      | 5000        |
| train/                  |             |
|    approx_kl            | 0.009534828 |
|    clip_fraction        | 0.0389      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.37       |
|    explained_variance   | -0.0122     |
|    learning_rate        | 0.0003      |
|    loss                 | 687         |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.00877    |
|    value_loss           | 1.14e+03    |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95.6     |
|    ep_rew_mean     | -167 

In [14]:
MAX_STEPS = 1_000

observation, info = env.reset(seed=42)
frames = []
step_count = 0

# for step in range(MAX_STEPS):
while True:
    frame = env.render()
    frames.append(frame)

    action, _ = model.predict(observation, deterministic=True)
    observation, reward, terminated, truncated, info = env.step(action)
    step_count += 1

    if terminated or truncated:
        print(f"Episode finished after {step_count} steps ({"truncated" if truncated else "terminated"})")
        break

env.close()

print(f"Final Step: {step_count}")
print(f"Number of Frames: {len(frames)}")

Episode finished after 602 steps (terminated)
Final Step: 602
Number of Frames: 602


In [15]:
from IPython.display import HTML
from base64 import b64encode
import os

# Create a video from the frames
video_filename = "../videos/lunarlander_ppo.mp4"
compressed_path = "../videos/lunarlander_ppo_compressed.mp4"
height, width, _ = frames[0].shape

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video = cv2.VideoWriter(video_filename, fourcc, 30.0, (width, height))

for frame in frames:
    video.write(
        cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    )
video.release()

print(f"Video guardado como {video_filename}")

os.system(f"rm {compressed_path}")
# Compressed video path
os.system(f"ffmpeg -i {video_filename} -vcodec libx264 {compressed_path}")
os.system(f"rm {video_filename}")
os.system(f"mv {compressed_path} {video_filename}")

# Show video
mp4 = open(video_filename, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(
    """
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>"""
    % data_url
)

Video guardado como ../videos/lunarlander_ppo.mp4


rm: cannot remove '../videos/lunarlander_ppo_compressed.mp4': No such file or directory
ffmpeg version n7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.1 (GCC) 20250207
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libdvdnav --enable-libdvdread --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enable-librsvg --enable-librubberband --enable-libsnappy --enab